This notebook follows the <a href="$./Prepare_Data">Prepare_Data notebook</a>. In this notebook, we will create an optimization model to determine which LEGO sets can be created from the collection (loose pile) of LEGO parts available.  

## Set up and initialize

### Import necessary packages

The optimization model will be implemented using Gurobipy (Python API of Gurobi).  We need to make sure that Gurobipy is installed and ready to be used.

In [0]:
%pip install gurobipy==12.0.0
dbutils.library.restartPython()

In [0]:
# Package imports

# Gurobi optimization library for model implementation
from gurobipy import GRB 
import gurobipy as gp

# Importing essential modules from pyspark.sql for DataFrame and Spark session management
from pyspark.sql import Row, SparkSession
import pyspark.pandas as ps
import pandas as pd
import numpy as np

# Filter warnings
import warnings
from pyspark.pandas.utils import PandasAPIOnSparkAdviceWarning
warnings.filterwarnings('ignore', category=PandasAPIOnSparkAdviceWarning)

### Load input data from Catalog

With our imports set, the next step is to load the data required for the model. We'll use the data previously processed into tables in the Prepare_Data notebook as inputs.

In [0]:
# Create a widget to accept a catalog name
dbutils.widgets.text('catalog_name','catalog_name','Enter catalog name')
dbutils.widgets.text('schema_name','schema_name','Enter schema name')

In [0]:
# Retrieve the catalog name
catalog_name = dbutils.widgets.get('catalog_name')
schema_name = dbutils.widgets.get('schema_name')
print(catalog_name + "/" + schema_name)

In [0]:
# Read in all the necessary data tables

# my_sets lists the original sets from which our pile of loose parts was created
my_sets = ps.read_table(f"{catalog_name}.{schema_name}.my_sets")
# relevant_set lists the number of parts of each type and color needed to build sets in my_sets
relevant_set = ps.read_table(f"{catalog_name}.{schema_name}.relevant_set")
# lego_sets table lists all possible LEGO sets
lego_sets = ps.read_table(f"{catalog_name}.{schema_name}.sets")
# parts table details all possible loose parts
parts = ps.read_table(f"{catalog_name}.{schema_name}.parts")
# colors table details of all possible colors
colors = ps.read_table(f"{catalog_name}.{schema_name}.colors")
# parts_avail table defines the loose parts we have at hand
parts_avail = ps.read_table(f"{catalog_name}.{schema_name}.parts_avail")
# parts_in_set defines the parts required to build a set in my_sets
part_in_set = ps.read_table(f"{catalog_name}.{schema_name}.part_in_set")

## Basic Optimization Model

Now we're ready to create an optimization model that can return the list of LEGO sets we can make from available parts that generate the most value.

Optimization models have a number of fundamental components that are present in all models:
1. **Input parameters**: The input data that informs the context of the optimization problem we are trying to solve.
2. **Decision variables**: The choices we have or the decisions we need to make.
3. **Constraints**: Restrictions or conditions that the solution must satisfy in order to be considered valid. These could be business rules or operating restrictions or resource limitations. 
4. **Objective(s)**: An expression that represents the goal to maximize or minimize. This could be an operational goal (maximize efficiency or minimize inventory), a business goal (maximize profit or maximize revenue) or a combination of both. An optimization problem can involve multiple goals or objective functions that must be balanced simultaneously.

For our LEGO problem, examples of these at a high level are:
1. **Input parameters**: Types of complete sets we could make, number and types of parts needed to make a complete set, number and types of parts on hand in inventory etc.
2. **Decision variables**: Which LEGO sets are we going to make with the parts available
3. **Constraints**: A solution cannot attempt to use more LEGO parts of each type than we have available. A solution must have all the required parts for the sets we chose to make in the right quantities.
4. **Objective**: Maximize the value of LEGO sets we can select.

We will introduce each of these components, followed by its implementation in Python/Gurobipy. 

NOTE: In the discussion of optimization models, readers will often encounter another component called **Sets**. Sets are primarily used for mathematical notation to help simplify the description of the mathematical model and ensure a model can be implemented independently of the datasets used to solve it. This allows us to easily solve the same mathematical model for different datasets. In the example below, we will illustrate the use of sets as well.

## Optimization Components
We will introduce the key components of any optimization model with a basic implementation below.

### Sets 

In mathematical programming (optimization), **sets** help define the scope of decision variables, constraints, and objective(s). For instance, in our example, sets are used to define parts, LEGO sets, color options, etc. Sets provide a structured way to group related entities so that we can scale our model easily whether we're solving a problem with 10 LEGO parts or 10,000 parts.

**Benefit:**
Most input data used in optimization models are defined over sets (with exception of scalar values).  Sets help to define scalable structure - instead of manually referencing each parameter, you can use sets to systematically iterate over inputs, variables, and constraints to represent each element in the set.  This makes the formulation and code implementation cleaner and easier to understand. More importantly, the notation allows us to decouple implementation of the optimization model from the underlying input datasets: you can use the same optimization model/code for different datasets by just changing the path to the input tables!

**Sets in the model**
- \\( p \in \text{part} \\): part ID, e.g. '60474' (Plate Round ...), '60169' (Minifig Chain 16L)
- \\( s \in \text{lego\textunderscore set} \\): set ID, e.g. '75524-1', '76078-1'
- \\( c \in \text{color} \\): color ID, e.g. '378' (Sand Green), '182' (Trans-Orange)
- \\( (p, c) \in \text{part\textunderscore in\textunderscore color} \\): part and color combination that exists in inventory, e.g. (3708, 4) is a valid part_num/color_id combination
- \\( (s, p, c) \in \text{part\textunderscore in\textunderscore set} \\): part of a specific color is used in set \\(s\\)


From the example below, sets can also be thought of as a collection of relevant columns from input datasets for our optimization model, written in a scalar form.

In [0]:
# Create sets for the optimization model from the data
# Note that all the variables created in this cell below are mathematical sets as described above

# Create a set of unique part numbers 
s_part = set(
    parts['part_num'].to_numpy()
    )
             
# Create a set of unique LEGO set numbers 
s_lego_set = set(
    lego_sets['set_num'].to_numpy()
    )

# Create a set of unique color IDs 
s_color = set(
    colors['id'].to_numpy()
    )

# Create a set of (part_num, color_id) pairs representing parts and their associated colors

s_part_in_color = set(zip(
    part_in_set['part_num'].to_numpy(), 
    part_in_set['color_id'].to_numpy()
    ))


# Create a set of (set_num, part_num, color_id) tuples indicating which parts and colors are included in each set
s_part_in_set = set(zip(
    part_in_set['set_num'].to_numpy(),
    part_in_set['part_num'].to_numpy(),
    part_in_set['color_id'].to_numpy()
))

### Parameters

Parameters refers to the input data that is used in the model formulation. These are values that influence the decision-making process but are not controlled by the optimization model. For example, the value of a LEGO set is a parameter, and it stays constant within an optimization run or a scenario. These input parameters can be static values or outputs from a predictive/ML model. For example, future demand predicted from a forecasting model can be used as an input for determining the right inventory to carry at various locations using an optimization model. 

Some input data are not indexed over a set, and can be defined by a **scalar** parameter instead.  For example, we can define a scalar parameter to specify a maximum limit on the number of LEGO sets to create from the loose parts. 

In order to test the robustness of a solution, scenario analysis is typically done by adjusting input parameters for different optimization runs, so that the impact on the objective or optimal decision variables can be evaluated. 

Python dictionaries provide a convenient way to store and retrieve input data associated with the model. The keys usually represent elements from previously defined sets, and values are typically numerical parameters.


**Parameters in the optimization model**

- \\( \text{part\textunderscore quantity \textunderscore available}_{p, c} \\): parts available for assortment e.g. {(3708, 4):1} indicates that we have 1 part available with part ID = 3708 and color ID = 4 
- \\( \text{part\textunderscore quantity\textunderscore in\textunderscore set}_{s, p, c} \\): quantity of parts of a specific color in a LEGO set
- \\( \text{set\textunderscore num\textunderscore value}_{s} \\): value of each LEGO set in the model 


We will create a function to convert data tables into dictionaries.

In [0]:
def ps_to_dict(df_ps, column_key, column_value):
    """
    Converts a pandas-on-Spark DataFrame to a dictionary.
    
    Parameters:
    df_ps (ps.DataFrame): The pandas-on-Spark DataFrame.
    column_key (str or list of str): Column(s) to use as the dictionary key(s).
    column_value (str): Column to use as the dictionary value.
    
    Returns:
    dict: A dictionary where keys are derived from `column_key` and values from `column_value`.
    """

    # Single column key
    if isinstance(column_key, str):
        dictionary = dict(zip(
            df_ps[column_key].to_numpy(), 
            df_ps[column_value].to_numpy()
            ))

    # Multi-column key
    elif isinstance(column_key, list):
        
        dictionary = dict(zip(
            zip(*(df_ps[col].to_numpy() for col in column_key)),
            df_ps[column_value].to_numpy()
        ))
    
    else:
        raise ValueError("column_key must be a string or a list of strings.")
    
    return dictionary


Now we'll use the function to create necessary input parameters.

In [0]:
# Define the parameters for the optimization model

# number of parts available
p_part_quantity_available = ps_to_dict(
    parts_avail,
    ["part_num", "color_id"],
    "quantity"
)

display(p_part_quantity_available)

# number of parts in each LEGO set per specific part type
p_part_quantity_in_set = ps_to_dict(
    part_in_set,
    ["set_num", "part_num", "color_id"],
    "quantity"
)

display(p_part_quantity_in_set)

# part parameters
p_part_name = ps_to_dict(
    parts,
    'part_num',
    'name'
)

display(p_part_name)
    
# LEGO set parameters 
p_set_num_part = ps_to_dict(
    lego_sets,
    'set_num',
    'num_parts'
)     

display(p_set_num_part)

# set the quantity of all other parts to zero 
temp_dict = {
    (p, c):0
        for (p, c) in s_part_in_color
        if (p, c) not in p_part_quantity_available
}
p_part_quantity_available.update(temp_dict)


#### Objective Function
For this example, we assume that the value of a set is proportional to the number of parts in the set. Thus, a set that has more parts is considered to be more desirable.

In [0]:

# value of each set in the objective
## assume set value is proportional to the number of parts in the set
p_set_num_value = {key: value for key, value in p_set_num_part.items()}

We have all the input parameters needed in the optimization model. We can now start implementing the model using Gurobi.

### Gurobi Settings

A Gurobi environment is a Gurobi object that provides the interface between your optimization model and the Gurobi solver. The environment 
manages important settings, resources, and connections required for solving optimization problems, including setting up the license.

In this example, we will keep the model size within Gurobi's free use limit.  So we don't need to set up a license in the environment. If you need to test larger models, a valid license needs to be procured from Gurobi.

In [0]:
# Initialize Gurobi environment
gp_env = gp.Env(empty=True)
# This line creates an empty Gurobi environment. By using empty=True, the environment is initialized without automatically loading parameters or settings, allowing for explicit customization before starting it.

gp_env.start()
#This line starts the environment, activating the connection between the optimization program and the Gurobi solver. This step is essential after creating an empty environment to ensure it is ready to use.

# Declare a Gurobi model using the specified Gurobi environment
model = gp.Model("base_lego_problem", env=gp_env)
#This line creates a new Gurobi model named "base_lego_problem", explicitly associating it with the previously initialized gp_env. By specifying the environment, you can customize settings (like logging or solver configurations) and reuse the same environment for other models if needed.

The next step is to set **Gurobi model options**, which control various aspects of the optimization model, including: 
- Setting time limit for model run
- Setting optimality tolerance level, which indicates the level of accuracy we want to solve the model to
- Gurobi specific settings to help the model solve to optimality faster through the use of [**heuristics**](https://assets.gurobi.com/pdfs/user-events/2017-frankfurt/MIP-Models-and-Heuristics.pdf)   

In [0]:
# Here are some commonly used Gurobi options we'll include in the optimization model to demonstrate their use.

GUROBI_OPTIONS = {
    "TimeLimit": 60,  # Sets a time limit for the optimization process (in seconds).
    "MIPGap": 0.01,   # Specifies the relative optimality gap (1% gap). In other words, Gurobi will terminate the search as soon as it founds any feasible solution within 1% value of the best possible solution. This option helps a user define their preferred tradeoff between computational effort (time) and the quality of the solution desired.
    "Threads": 8      # Limits the number of threads Gurobi uses for computation to 8.
}

# Set the Gurobi options above in the model declared in the previous cell
for param_name, param_val in GUROBI_OPTIONS.items():
    model.setParam(param_name, param_val)

### Decision Variables

Decision variables are a core element for an optimization model. They represent the choices or decisions that the model seeks to determine. For example, in this model, we're trying to determine which LEGO sets (out of all potential options) to choose.  In a retail assortment model, the decision variables would be which product to stock at each location, as well as the quantity of each product to stock.  

Decision variables can be either continuous (real number within a defined range) or binary (0 or 1).  Continuous variables are typically used to represent decisions that can vary smoothly, such as quantity, inventory level, or resource consumption; whereas binary variables are used for discrete or on/off choices (e.g. whether to open a new store (1) at a location or not (0) , whether assign a job to a machine (1) or not (0)).





**Binary Variables**

This LEGO optimization model uses only binary decision variables.  We name the variable \\(bv^{\text{select\textunderscore set}}\\). The variable is indexed over LEGO sets, which means that all the LEGO sets in \\(s\\) are considered as choices in the selection. 

- \\(bv^{\text{select\textunderscore set}}_{s} \\): the output of the decision variable = 1 if the LEGO set \\(s\\) is selected, and is 0 if unselected to build

 

In [0]:
# Add binary decision variables for selecting LEGO sets
# addVars is a Gurobi function to add variables to the model.
# Each variable in bv_select_set corresponds to an element in s_lego_set.
bv_select_set = model.addVars(
    s_lego_set,             # the set of elements that the decision variable is indexed over
    vtype=GRB.BINARY,       # specifies that these variables are binary (0 or 1)
    name="bv_select_set"    # define a name for the variable
)

Note that we only need to create one symbolic binary variable to represent each LEGO set's selection decision.  We can do this by indexing the variable over the model set `s_lego_set`.  Internally, Gurobi will create a binary variable for each element of the `s_lego_set`, as if it was run in a for loop!

### Constraints

Constraints are mathematical expressions that define the rules, limits, and conditions an optimization model must follow. They ensure that the solutions generated by the model are feasible and align with user requirements.  

Some uses for constraints are: 
- Define feasiblity: time window for an action, capacity limit
- Real-world or physical rules: mass balance, logical conditions
- Improve computational speed: some constraints can be constructed to help the model reach optimality sooner by eliminating infeasible options or obviously bad solutions

Constraints take the form of either inequalities or equalities.

**Model Constraints**

In the LEGO example, we have a fundamental constraint, which is that the total number of parts used by selected LEGO sets has to be less than or equal to their availability in the loose pile.  Mathematically, it's written as follows:

$$ \sum_{s:(s,p,c)\in \text{part\textunderscore in\textunderscore set}} \text{part\textunderscore quantity\textunderscore in\textunderscore set}_{s,p,c} \times bv^{\text{select\textunderscore set}}_{s} \leq \text{part\textunderscore quantity \textunderscore available}_{p,c} \qquad \forall (p,c) \in \text{part\textunderscore in \textunderscore color}$$ 

For each valid part/color combination, the LHS (left hand side) of the inequality expresses the quantity of such part used by LEGO sets selected, and the RHS (right hand side) is the total number of such parts available in the loose pile.  The inequality sign enforces that we cannot use more parts than what's available.
The multiplication with the binary variable \\(bv^{\text{select\textunderscore set}}\\) ensures that the parts are only counted against our available limits if the set is selected to be built (\\(bv^{\text{select\textunderscore set}} = 1\\)).

In [0]:
# Define the constraint: the total quantity of a specific part (p) in a specific color (c) 
# used by the selected sets must not exceed the quantity available.

# Iterate over all part-color combinations
for (p, c) in s_part_in_color:

    # Create a unique name for each constraint for better identification in the model
    cons_name = f"c_part_quantity_balance[{p}-{c}]"
    
    constr = (
        sum(   # Define the left-hand side (LHS) of the constraint: Sum of the quantities of part (p) in color (c) across all selected sets (s).
            p_part_quantity_in_set[s, p, c] * bv_select_set[s]  # Quantity of (p, c) in set (s) times the binary variable
            for s in s_lego_set
            if (s, p, c) in s_part_in_set  # Only consider sets that actually contain the part-color combination
        ) 
        <= 
        p_part_quantity_available[p, c]  # Right-hand side (RHS): available quantity of part (p) in color (c)
    )
    
    # Add the constraint to the model with the generated name
    model.addConstr(constr, name=cons_name)


**Component breakdown:**

- Constraint Name:

  - `cons_name = f"c_part_quantity_balance[{p}-{c}]"` 
  - Assigns a meaningful name to the constraint for easier debugging and interpretation in solver output.

- LHS (Left-Hand Side):
  - `sum(...)` computes the total usage of the part-color combination \\((p,c)\\) across all selected sets.
  - The `p_part_quantity_in_set[s, p, c]` represents how many of part \\(p\\) in color \\(c\\) are in set \\(s\\).
  - The binary variable `bv_select_set[s]` ensures the contribution is counted only if set \\(s\\) is selected (when the value for `bv_select_set[s]` is 1)

- Filter Condition:
  - `if (s, p, c) in s_part_in_set` ensures that only sets containing the part-color combination \\((p,c)\\) are considered.

- RHS (Right-Hand Side):
  - `p_part_quantity_available[p, c]` specifies the maximum available quantity of part \\(p\\) in color \\(c\\) in the loose pile.

- Adding the Constraint:
  - `model.addConstr(constr, name=cons_name)` adds the constraint to the model so the solver enforces it during optimization.

### Objective

The objective refers to the function that needs to be optimized — either maximized or minimized. This is typically a mathematical expression that represents the goal of the optimization problem. Some examples of objective are to maximize profit, minimize cost, or minimize resources required.

It's possible that there could be more than one objective for a single optimization model, see [here](https://docs.gurobi.com/projects/optimizer/en/current/features/multiobjective.html) for additional information on how competing objectives can be modeled in the same formulation.

In the LEGO example, our objective is to maximize total value of the selected LEGO sets, and it is represented mathematically here:

   $$ \text{max} \sum_{s} \text{set\textunderscore value}_{s} \times bv^{\text{select\textunderscore set}}_{s} $$

In [0]:
# Calculate the objective function by summing the values of selected sets (p_set_num_value[s])
# If set 's' is selected (bv_select_set[s] = 1), its associated value (p_set_num_value[s]) contributes to the objective.
obj = sum(p_set_num_value[s]*bv_select_set[s] for s in s_lego_set)

# Set the model's objective function to maximize the total value calculated in 'obj'
# GRB.MAXIMIZE indicates that we want to find the maximum possible value for the objective function.
model.setObjective(obj, GRB.MAXIMIZE)

# Update the model to ensure that the objective and any other changes are reflected before solving.
model.update()

### Solve Model

The model is now complete, and it is considered an integer programming (IP) problem, or more specifically a binary programming (BP) problem. Since we do not permit a partially built set and each decision is a simple yes or no, all decision variables must be binary values (0, 1) which are a subset of integer values. 

The essential components of the optimization model have been stored in `model` object, now we can call Gurobi to trigger model solve to obtain the optimal solution.  Behind the scene, Gurobi searches for the optimal solution without having to test every single valid (feasible) combination of LEGO sets. It leverages approaches called Branch-and-Bound and Cutting Planes to identify optimal solutions efficiently.



In [0]:
# Optimize the model using the solver. 
model.optimize()

# After optimization, retrieve the value of the objective function from the model. 
# This stores the optimal value of the objective function (if an optimal solution exists) in 'objective_base'.
objective_base = model.ObjVal

# Check the model's status to ensure that an optimal solution was found.
if model.status == GRB.OPTIMAL:
    # If the model status indicates an optimal solution was found, display a success message.
    display("Optimal solution found.")
else:
    # If the model status indicates that the solution is not optimal (e.g., infeasible), raise an exception.
    raise Exception("Optimal solution not found.")

Above is the **solution log** from the Gurobi solve.  It provides key details on the optimization process:

- Model Size: The log displays the number of constraints, and variables to help assess model complexity. Example: `Optimize a model with 975 rows, 18 columns and 1357 nonzeros`.  'Rows' indicates number of constraints, and 'columns' indicates number of variables. 'Nonzeros' gives an indication of the sparsity of the constraint matrix.

- Optimization Progress: Gurobi shows the number of iterations `0 simplex iterations`, the optimality gap `gap 0.0000%`, and time spent `0.01 seconds`. Notice that for this problem size the solver immediately obtains a heuristic solution `Found heuristic solution: objective 619.0000000` but ultimately finds an alternative solution with a higher objective value.

- Solution Status: The status tells if the solution is optimal, infeasible, or unbounded. It also provides the objective value when an optimal solution is found. Example: `Optimal solution found.`

The next step is to review model solution.  We can contrast the optimal solution that Gurobi returned against the 4 LEGO sets that we used originally to create the loose parts inventory in the previous notebook.


In [0]:
# retrieves the solution values for the variables in bv_select_set from the Gurobi model
# since it's a binary variable, we just need to keep the sets are selected by the model (i.e. 
# if the result is 1)

bv_select_data = {
    var: val
    for var, val in model.getAttr('X', bv_select_set).items()
}

df_bv_select_data = pd.DataFrame(
    [
        {'Set': k, 'Value': v}
        for k, v in bv_select_data.items()
        if v > 0.01 # We are looking for elements where the value is 1 (not 0). We use a numerical tolerance to avoid floating point precision issues
    ])

display(df_bv_select_data)

The solution selects the three most valuable sets from the collection that was originally picked to create the part inventory data!
* 75160-1: U-wing
* 75168-1: Yoda's Jedi Starfighter
* 75170-1: The Phantom

Notice that the Y-wing set (75162-1) was not selected. This is because we perturbed the inventory by removing a single piece common to the U and Y-wing models. With the number of pieces for the U-wing being higher, its value -as defined above- is also higher, making it the optimal selection when maximizing overall value.


Lastly, we display the inventory of unused parts left over from the starting inventory. Since we had a missing part, there is clearly unused inventory left behind.

In [0]:
remaining_parts = { # Remaining parts
    (p, c): p_part_quantity_available[p, c] # We start with parts available
     - sum(                                 # We subtract parts that are used up in building selected sets
        p_part_quantity_in_set[s, p, c]*bv_select_set[s].X
            for s in s_lego_set
            if (s, p, c) in s_part_in_set
        )
    for (p, c) in s_part_in_color
}

# Create a dictionary of remaining parts where there are nonzero values available
remaining_parts_data = {                    
    (p, c): remaining_parts[p, c]
        for (p ,c) in s_part_in_color
        if remaining_parts[p, c] > 0
}

df_remaining_parts_data = pd.DataFrame(
    [{'Part': k[0], 'Color': k[1], 'Quantity': v} for k, v in remaining_parts_data.items()]
)

if len(remaining_parts_data) > 0:
    display(df_remaining_parts_data)
else:
    display("No remaining parts")

This wraps the small optimization example. We introduced key optimization concepts, defined a mathematical optimization model, implemented it in Gurobi and obtained an optimal solution from Gurobi.

The dataset used for this example was admittedly small. How does mathematical optimization fare on large scale datasets? We will investigate this question in the next notebook: <a href="$./Optimization_Model_Large">Optimization_Model_Large</a>. We will apply our optimization model against a non-trivial large scale dataset for which we would not be able to determine a manual solution easily. Additionally, we will compare optimization results against results from a greedy heuristic to truly showcase the power of optimization.

**NOTE: To test and run the larger problem definition, you will need to obtain a commercial Gurobi license as the dimension of the problem is higher than the limits of the restricted use license used in this notebook.**